# url 크롤링 후 피클로 저장

In [17]:
import pickle
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

def get_webtoon_urls_by_tab(tab_name, driver):
    """
    특정 요일 또는 '완결' 탭에서 '전체' 버튼을 클릭한 후, 모든 웹툰 URL을 가져오는 함수
    """
    base_url = f"https://webtoon.kakao.com/?tab={tab_name}"
    driver.get(base_url)
    time.sleep(3)  # 페이지 로딩 대기

    print(f"🔍 {tab_name.upper()} 탭 크롤링 시작...")

    # 🔹 "전체" 버튼 클릭 (버튼이 로드될 때까지 기다린 후 클릭)
    try:
        entire_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[span[text()='전체']]"))
        )
        entire_button.click()
        time.sleep(2)  # 클릭 후 페이지 변경 대기
    except Exception as e:
        print(f"❌ '{tab_name.upper()}' 탭에서 '전체' 버튼을 찾을 수 없거나 클릭할 수 없습니다.", e)

    # 🔹 무한 스크롤 (스크롤을 끝까지 내려서 모든 웹툰 로드)
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)  # 페이지 끝으로 스크롤
        time.sleep(2)  # 데이터 로딩 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:  # 더 이상 스크롤이 내려가지 않으면 종료
            break
        last_height = new_height

    # 🔹 웹툰 목록에서 URL 가져오기
    webtoon_urls = []
    webtoon_elements = driver.find_elements(By.CSS_SELECTOR, "a[href*='/content/']")  # 웹툰 링크 요소 찾기
    
    for element in webtoon_elements:
        webtoon_url = element.get_attribute("href")  # 웹툰 URL 가져오기
        if webtoon_url and webtoon_url.startswith("https://webtoon.kakao.com/content/"):
            webtoon_urls.append(webtoon_url)

    print(f"✅ {tab_name.upper()} 탭에서 {len(webtoon_urls)}개의 웹툰을 수집했습니다.\n")
    return list(set(webtoon_urls))  # 중복 제거 후 반환


def get_all_webtoon_urls():
    """
    Selenium을 사용하여 모든 요일 및 완결 탭에서 웹툰 URL을 가져오고 피클 파일로 저장하는 함수
    """
    # 🔹 Chrome WebDriver 설정
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # GUI 없이 실행 (백그라운드 모드)
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # 🔹 요일 + 완결 탭 목록
    tabs = ["mon", "tue", "wed", "thu", "fri", "sat", "sun", "complete"]
    all_webtoon_urls = []

    for tab in tabs:
        urls = get_webtoon_urls_by_tab(tab, driver)
        all_webtoon_urls.extend(urls)

    driver.quit()  # 브라우저 종료
    

    all_webtoon_urls = list(set(all_webtoon_urls))  # 중복 제거

    # 🔹 웹툰 URL 리스트를 피클 파일로 저장
    with open("kakao_webtoon_urls.pkl", "wb") as f:
        pickle.dump(all_webtoon_urls, f)

    print(f"✅ 웹툰 URL이 'kakao_webtoon_urls.pkl' 파일로 저장되었습니다.")
    
    return all_webtoon_urls

# ✅ 크롤링 실행 및 저장
webtoon_list = get_all_webtoon_urls()

# ✅ 피클 파일에서 데이터 불러오기 테스트
with open("kakao_webtoon_urls.pkl", "rb") as f:
    loaded_webtoon_list = pickle.load(f)

print(f"📂 저장된 웹툰 URL 개수: {len(loaded_webtoon_list)}개")
for url in loaded_webtoon_list[:10]:  # 상위 10개만 출력
    print(url)


🔍 MON 탭 크롤링 시작...


KeyboardInterrupt: 

# 카카오웹툰 자동 로그인

In [11]:
import os
import re
import time
import json
from bs4 import BeautifulSoup as bs
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

load_dotenv()
# WebDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

kakao_id = os.getenv("KAKAO_ID")
kakao_pw = os.getenv("KAKAO_PWD")
driver.get("https://webtoon.kakao.com/more")

# 로그인처리
login_icon = r"#root > main > div > div.bg-background-02 > div > div > a:nth-child(1) > div > img.w-113.h-38"
driver.find_element(By.CSS_SELECTOR, login_icon).click()
time.sleep(2)
login_button = r"body > div:nth-child(5) > div > div > div > div.overflow-x-hidden.overflow-y-auto.\!overflow-hidden.flex.flex-col > div.text-center.pb-\[112px\].overflow-y-auto > div > div > button"
driver.find_element(By.CSS_SELECTOR, login_button).click()
time.sleep(2)

ID_selector = r"#loginId--1"
PWD_selector = r"#password--2"
ID_input = driver.find_element(By.CSS_SELECTOR, ID_selector)
ActionChains(driver).send_keys_to_element(ID_input, kakao_id).perform()
PWD_input = driver.find_element(By.CSS_SELECTOR, PWD_selector)
ActionChains(driver).send_keys_to_element(PWD_input, kakao_pw).perform()
login_selector = (
    r"#mainContent > div > div > form > div.confirm_btn > button.btn_g.highlight.submit"
)
driver.find_element(By.CSS_SELECTOR, login_selector).click()
time.sleep(2)
# 동시접속 기기 횟수 초과 시

alert = r"body > div:nth-child(8) > div > div > div > div.absolute.w-full.px-18.bottom-0.pb-30.flex.left-0.z-1.bg-grey-01.light\:bg-white.Alert_buttonsWrap__2ln9d.pt-10 > button.relative.px-10.py-0.btn-white.light\:btn-black.button-active"
driver.find_element(By.CSS_SELECTOR, alert).click()


# 웹툰 정보 가져오기

In [ ]:
import pickle
import requests
import json
import re
import time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

def extract_webtoon_id(url):
    """카카오웹툰 URL에서 웹툰 ID를 추출하는 함수"""
    match = re.search(r'/content/.+/(\d+)', url)
    return match.group(1) if match else None

def get_webtoon_price_selenium(webtoon_page_url):
    """Selenium을 사용하여 웹툰의 가격 정보를 가져오는 함수"""
    time.sleep(4)
    driver.get(webtoon_page_url)
    time.sleep(2)

    soup = bs(driver.page_source, "html.parser")

    # 🔹 가격 정보가 있는 배지 찾기
    badges = soup.find_all("div", class_="flex flex-wrap gap-4 mb-12")

    for badge in badges:
        texts = badge.find_all("p", class_=lambda x: x and "whitespace-pre-wrap" in x and "font-badge" in x)
        for text in texts:
            text = text.text.strip()

            # 🔹 가격 정보 확인
            if "마다 무료" in text:
                return "기다리면 무료"
            elif "연재" in text:
                return "무료"

    # 🔹 가격 정보가 없으면 기본값 "유료" 반환
    return "유료"

# 숫자 변환
def convert_views(view_text):
    if "만" in view_text:
        return int(float(view_text.replace(",", "").replace("만", "")) * 10000)
    elif "억" in view_text:
        return int(float(view_text.replace(",", "").replace("억", "")) * 100000000)
    else:
        return int(view_text.replace(",", "")) if view_text.replace(",", "").isdigit() else "-"

def get_webtoon_info(webtoon_id):
    """API를 사용해 웹툰 기본 정보를 가져오고, Selenium을 사용해 추가 정보를 크롤링하는 함수"""
    profile_url = f"https://gateway-kw.kakao.com/decorator/v2/decorator/contents/{webtoon_id}/profile"
    episode_url = f"https://gateway-kw.kakao.com/episode/v2/views/content-home/contents/{webtoon_id}/episodes?sort=-NO&offset=0&limit=30"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
        "Referer": "https://webtoon.kakao.com/",
        "Accept-Language": "ko",
    }

    # ✅ **Selenium으로 API URL 직접 열기**
    time.sleep(2)
    driver.get(profile_url)
    time.sleep(3)  # 페이지 로딩 대기

    # HTML에서 JSON 추출
    soup = bs(driver.page_source, "html.parser")
    time.sleep(2)
    pre_tag = soup.find("pre")  # API 응답이 JSON 형태로 담겨 있음
    if not pre_tag:
        print(f"❌ API 응답을 찾을 수 없음: {webtoon_id}")
        return None

    # JSON 파싱
    try:
        data = json.loads(pre_tag.text)  # Selenium에서 가져온 데이터를 JSON으로 변환
    except json.JSONDecodeError:
        print(f"❌ JSON 디코딩 실패: {webtoon_id}")
        return None

    # 웹툰 정보 파싱
    seo_id = data.get("data", {}).get("seoId", "")
    webtoon_page_url = f"https://webtoon.kakao.com/content/{seo_id}/{webtoon_id}?tab=profile"

    # 작가, 일러스트레이터, 원작 분리
    author, illustrator, original = "-", "-", "-"
    for person in data.get("data", {}).get("authors", []):
        if person.get("type") == "AUTHOR":
            author = person.get("name", "-")
        elif person.get("type") == "ILLUSTRATOR":
            illustrator = person.get("name", "-")
        elif person.get("type") == "ORIGINAL_STORY":
            original = person.get("name", "-")

    # 연재 상태 설정
    status = "미정"
    status_map = {
        "COMPLETED": "완결",
        "END_OF_SEASON": "연재",
        "SEASON_COMPLETED":"연재",
        "EPISODES_PUBLISHING": "연재",
        "EPISODES_NOT_PUBLISHING": "휴재"
    }

    for badge in data.get("data", {}).get("badges", []):
        if badge.get("type") == "STATUS":
            status = status_map.get(badge.get("title"), "미정")

    # 연재 요일 변환
    weekdays_map = {
        "MON": "월요일", "TUE": "화요일", "WED": "수요일",
        "THU": "목요일", "FRI": "금요일", "SAT": "토요일", "SUN": "일요일"
    }
    update_days = "-"
    if status not in ["휴재", "완결"]:
        update_days = [
            weekdays_map.get(b.get("title", "").upper(), "-")
            for b in data.get("data", {}).get("badges", []) if b.get("type") == "WEEKDAYS"
        ]
        update_days = ", ".join(update_days) if update_days else "-"

    # 연령 제한
    age_rating = "전체 이용가"
    for badge in data.get("data", {}).get("badges", []):
        if badge.get("type") == "AGE_LIMIT":
            age_rating = f"{badge.get('title', '')}세 이용가"

    # 키워드 정리
    keywords = ", ".join([k.replace("#", "") for k in data.get("data", {}).get("seoKeywords", [])])

    # 가격 정보 설정 (기본값 "-")
    price = "-"

    for badge in data.get("data", {}).get("badges", []):
        if badge.get("type") == "INFO":
            if badge.get("title") == "WAIT_FOR_FREE":
                price = "기다리면 무료"
            elif badge.get("title") == "FREE_PUBLISHING":
                price = "무료"

    # 유료 웹툰인 경우 Selenium으로 가격 크롤링
    if price == "-":
        price = get_webtoon_price_selenium(webtoon_page_url)

    # API에서 에피소드 개수 가져오기
    episode_response = requests.get(episode_url, headers=headers)
    episode_count = episode_response.json().get("meta", {}).get("pagination", {}).get("totalCount", 0) if episode_response.status_code == 200 else 0

    # Selenium을 이용한 추가 정보 크롤링
    time.sleep(3)
    driver.get(webtoon_page_url)
    time.sleep(2)

    soup = bs(driver.page_source, "html.parser")

    # 장르, 조회수, 좋아요 가져오기
    genre, views, likes = "-", "-", "-"
    stats = soup.find_all("div", class_="flex justify-center items-start h-14 mt-8 leading-14")

    for stat in stats:
        items = stat.find_all("p", class_="whitespace-pre-wrap break-all break-words support-break-word s12-regular-white ml-2 opacity-75")
        if len(items) >= 3:
            genre = items[0].text.strip()  # 장르
            views = convert_views(items[1].text.strip())  # 조회수
            likes = convert_views(items[2].text.strip())  # 좋아요

    # 웹툰 정보 정리
    webtoon_info = {
        "id": int(webtoon_id),
        "type": "웹툰",
        "platform": "카카오웹툰",
        "title": data.get("data", {}).get("title", ""),
        "status": status,
        "update_days": update_days,
        "thumbnail": "-",
        "genre": genre,
        "views": views,
        "rating": "-",
        "likes": likes,
        "synopsis": data.get("data", {}).get("synopsis", ""),
        "keywords": keywords,
        "author": author,
        "illustrator": illustrator,
        "original": original,
        "age_rating": age_rating,
        "price": price,
        "episode": episode_count,
        "url": webtoon_page_url
    }
    return webtoon_info


In [ ]:
# 피클 파일에서 웹툰 URL 불러오기
with open("webtoon_kko_urls.pkl", "rb") as f:
    webtoon_urls = pickle.load(f)

with open("kakao_webtoon_data.json", "r", encoding="utf-8") as f:
    total = json.load(f) # <- 기존 크롤링한 데이터 불러오기

# 웹툰 정보 크롤링 실행
webtoon_data_list = []
for url in webtoon_urls[2380:2400]:  # 할만큼 지정
    webtoon_id = extract_webtoon_id(url)
    if webtoon_id:
        webtoon_data = get_webtoon_info(webtoon_id)
        if webtoon_data:
            webtoon_data_list.append(webtoon_data)

total += webtoon_data_list # <- 데이터 합치기

# JSON 파일로 저장
with open("kakao_webtoon_data.json", "w", encoding="utf-8") as f:
    json.dump(total, f, ensure_ascii=False, indent=4) # <- 총 데이터 json으로 내보내기

print(f"총 {len(webtoon_data_list)}개의 웹툰 정보를 저장했습니다.")


✅ 총 20개의 웹툰 정보를 저장했습니다.


In [ ]:
import json
import glob

# JSON 파일 리스트 가져오기 (현재 디렉토리의 모든 JSON 파일)
json_files = glob.glob("kakao_webtoon_data/*.json")

# 데이터를 저장할 리스트 초기화
merged_data = []

# 모든 JSON 파일을 읽어서 리스트에 추가
for file in json_files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)  # JSON 파일 로드
        if isinstance(data, list):  # 데이터가 리스트 형태인지 확인
            merged_data.extend(data)  # 리스트 확장
        else:
            print(f"{file} 파일은 리스트 형식이 아닙니다. 건너뜁니다.")

# 중복 제거 (ID 기준)
merged_data = {item["id"]: item for item in merged_data}.values()

# 합친 데이터를 새로운 JSON 파일로 저장
with open("webtoon_kakao_crawling.json", "w", encoding="utf-8") as f:
    json.dump(list(merged_data), f, ensure_ascii=False, indent=4)

print(f"{len(json_files)}개의 JSON 파일을 병합했습니다.")


✅ 49개의 JSON 파일을 병합했습니다.


In [7]:
import pickle
import json

# 대상 웹툰 ID 리스트
target_ids = {753, 587, 4325, 894, 4184, 3837, 3698, 998, 3411}

# 피클 파일에서 웹툰 URL 불러오기
with open("webtoon_kko_urls.pkl", "rb") as f:
    webtoon_urls = pickle.load(f)

# 기존 JSON 파일 로드
with open("webtoon_kko_crawling.json", "r", encoding="utf-8") as f:
    total = json.load(f)  # 기존 크롤링 데이터 불러오기

# ID 기준으로 기존 데이터 딕셔너리 변환 (빠른 조회를 위해)
total_dict = {item["id"]: item for item in total}

# 대상 ID에 해당하는 URL 필터링
filtered_urls = [url for url in webtoon_urls if extract_webtoon_id(url) and int(extract_webtoon_id(url)) in target_ids]

# 웹툰 정보 크롤링 실행 (대상 ID만)
for url in filtered_urls:
    webtoon_id = extract_webtoon_id(url)
    if webtoon_id:
        webtoon_data = get_webtoon_info(webtoon_id)
        if webtoon_data:
            total_dict[int(webtoon_id)] = webtoon_data  # 기존 데이터 업데이트

# 업데이트된 데이터를 다시 리스트로 변환
updated_total = list(total_dict.values())

# JSON 파일로 저장
with open("webtoon_kko_crawling_1.json", "w", encoding="utf-8") as f:
    json.dump(updated_total, f, ensure_ascii=False, indent=4)

print(f"총 {len(filtered_urls)}개의 웹툰 정보를 다시 크롤링하여 업데이트했습니다.")


총 9개의 웹툰 정보를 다시 크롤링하여 업데이트했습니다.


In [13]:
import json
import time
import re
import random
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ✅ Selenium WebDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# ✅ 1️⃣ JSON 파일 로드
with open("webtoon_kko_crawling_1.json", "r", encoding="utf-8") as f:
    total = json.load(f)  # 기존 크롤링 데이터 불러오기

# ✅ 2️⃣ "유료"인 웹툰만 필터링
paid_webtoons = [webtoon for webtoon in total if webtoon["price"] == "유료"]

# ✅ 3️⃣ 가격 정보 업데이트 함수
def update_webtoon_price(webtoon):
    webtoon_url = webtoon["url"]
    episode_count = webtoon["episode"]

    try:
        driver.get(webtoon_url)  # 웹툰 페이지 접속
        time.sleep(random.uniform(3, 5))  # 🚀 3~5초 랜덤 대기 (403 Forbidden 방지)

        # 🔹 페이지 로딩 확인
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

        soup = bs(driver.page_source, "html.parser")

        # 🔹 "연재무료" 배지 확인
        free_badge = soup.find("p", class_="whitespace-pre-wrap break-all break-words support-break-word font-badge !whitespace-nowrap rounded-5 s10-bold-darkGrey01 bg-kakaopage-yellow px-5 !text-[11px]")
        if free_badge and "연재무료" in free_badge.text:
            webtoon["price"] = "무료"
            print(f"✅ {webtoon['title']} - 연재무료 확인 → 무료 변경")
            time.sleep(random.uniform(2, 4))  # 🚀 추가 대기
            return webtoon  # 업데이트 후 반환

        # 🔹 "XX편 무료" 배지 확인
        free_episode_badge = soup.find("p", class_="whitespace-pre-wrap break-all break-words support-break-word font-badge !whitespace-nowrap rounded-5 s10-bold-black bg-white px-5 !text-[11px]")
        if free_episode_badge:
            free_episode_text = free_episode_badge.text.strip()
            match = re.search(r"(\d+)편 무료", free_episode_text)
            if match:
                free_episode_count = int(match.group(1))
                if free_episode_count == episode_count:
                    webtoon["price"] = "무료"
                    print(f"✅ {webtoon['title']} - {free_episode_count}편 무료 → 무료 변경")
                    time.sleep(random.uniform(2, 4))  # 🚀 추가 대기

        return webtoon  # 업데이트 후 반환

    except Exception as e:
        print(f"❌ {webtoon['title']} 크롤링 중 오류 발생: {e}")
        return webtoon  # 오류 발생 시 원본 데이터 반환

# ✅ 4️⃣ 필터링된 웹툰의 가격 정보 업데이트
updated_webtoons = [update_webtoon_price(webtoon) for webtoon in paid_webtoons]

# ✅ 5️⃣ 기존 데이터에 반영
webtoon_dict = {w["id"]: w for w in total}  # 기존 데이터 딕셔너리로 변환
for updated_webtoon in updated_webtoons:
    webtoon_dict[updated_webtoon["id"]] = updated_webtoon  # 기존 데이터 업데이트

# ✅ 6️⃣ 업데이트된 데이터 다시 JSON 파일로 저장
updated_total = list(webtoon_dict.values())

with open("webtoon_kko_crawling_1_updated.json", "w", encoding="utf-8") as f:
    json.dump(updated_total, f, ensure_ascii=False, indent=4)

print(f"✅ 총 {len(paid_webtoons)}개의 웹툰 가격 정보를 업데이트했습니다.")


KeyboardInterrupt: 